In [ ]:
#index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Text to Speech Converter</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="container">
        <h1>Text to Speech Converter</h1>
        <textarea id="textInput" placeholder="Enter text here..."></textarea>
        
        <div class="buttons">
            <select id="voiceSelect">
              <option value="female">Female</option>
            </select>
            <button onclick="speakText()">Play</button>
            <button onclick="downloadAudio()">Download</button>
        </div>
    </div>

    <script src="{{ url_for('static', filename='script.js') }}"></script>
</body>
</html>

In [ ]:
#style.css

/* Full Page Background */
body {
    background-image: url("background.jpg"); /* Static folder la iruntha auto load aagum */
    background-size: cover;
    background-position: center;
    text-align: center;
    font-family: Arial, sans-serif;
    color: white;
}

.container {
    margin-top: 100px;
}

textarea {
    width: 80%;
    height: 150px;
    font-size: 18px;
    padding: 10px;
    border-radius: 5px;
    border: none;
    resize: none;
}

.buttons {
    margin-top: 20px;
}

button, select {
    padding: 10px 20px;
    margin: 5px;
    font-size: 18px;
    border-radius: 5px;
    border: none;
    cursor: pointer;
}

button {
    background-color: #28a745;
    color: white;
}

button:hover {
    background-color: #218838;
}

In [ ]:
#script.js
function speakText() {
    let text = document.getElementById("textInput").value;
    let voiceSelect = document.getElementById("voiceSelect").value;

    if (!text) {
        alert("Please enter text to convert.");
        return;
    }

    let speech = new SpeechSynthesisUtterance(text);
    let voices = window.speechSynthesis.getVoices();
    
    // Voice selection based on male/female
    if (voiceSelect === "female") {
        speech.voice = voices.find(voice => voice.name.includes("Female")) || voices[0];
        speech.pitch = 1.5;  // Female voice high pitch
    } else {
        speech.voice = voices.find(voice => voice.name.includes("Male")) || voices[0];
        speech.pitch = 1;  // Male voice deep pitch
    }

    speech.lang = "en-US";
    speech.rate = 1;
    
    window.speechSynthesis.speak(speech);
}

function downloadAudio() {
    let text = document.getElementById("textInput").value;
    let voice = document.getElementById("voiceSelect").value;

    if (!text) {
        alert("Please enter text to download.");
        return;
    }

    fetch('/download_audio', {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify({ text: text, voice: voice })
    })
    .then(response => response.blob())
    .then(blob => {
        let url = window.URL.createObjectURL(blob);
        let a = document.createElement("a");
        a.href = url;
        a.download = "speech.mp3";
        document.body.appendChild(a);
        a.click();
        document.body.removeChild(a);
    })
    .catch(error => console.error("Error:", error));
}

In [ ]:
#python
from flask import Flask, render_template, request, send_file, jsonify
from gtts import gTTS
import os
import pydub
import tempfile

app = Flask(__name__)

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/download_audio", methods=["POST"])
def download_audio():
    data = request.json
    text = data.get("text", "")
    voice = data.get("voice", "")

    if not text:
        return jsonify({"error": "No text provided"}), 400

    audio_path = "static/speech.mp3"

    if voice == "female":
        tts = gTTS(text, lang="en")
        tts.save(audio_path)
    else:
        temp_path = tempfile.mktemp(suffix=".mp3")
        tts = gTTS(text, lang="en")
        tts.save(temp_path)

        sound = pydub.AudioSegment.from_file(temp_path, format="mp3")
        sound.export(audio_path, format="mp3")

    return send_file(audio_path, as_attachment=True, mimetype="audio/mp3")
if __name__ == "__main__":
    app.run(debug=True)